# ASR Baseline experiment using Whisper and Europarl-ST (Spanish to English)

In this notebook, we are going to learn how to use the Open AI pre-trained model [Whisper](https://openai.com/index/whisper/) for ASR on the [Europarl-ST dataset](https://huggingface.co/datasets/tj-solergibert/Europarl-ST) (using the Spanish-English speech data).

First, we import some OpenAI source whisper libraries and additional ones (e.g. for computing Word Error Rate, WER)

In [ ]:
import whisper
from whisper.normalizers import BasicTextNormalizer

from tqdm.notebook import tqdm
import pandas as pd

import jiwer

model = whisper.load_model("base")

Load Europarl-ST speech dataset

In [2]:
audios = []

with open(r"./Europarl-ST/es/en/test/Europarl-ST.v2.es.en.test.200.lst", "r", encoding="utf-8") as lista_audios:
    for linea in lista_audios:
        audios.append(str(linea).strip())

len(audios)

86

<p style="page-break-after:always;"></p>

Transcribe all the audio data using the Whisper (base) model. The ASR output is stored in hypotheses. At the same time, transcription references are stored into references and translation references into translations.

In [3]:
hypotheses = []
references = []
translations = []

for audio in tqdm(audios):
    with open(r"./Europarl-ST/es/en/test/%s/transcription.tok" % audio, "r", encoding="utf-8") as referencia:
        references.append(referencia.read())
    with open(r"./Europarl-ST/es/en/test/%s/translation_en" % audio, "r", encoding="utf-8") as translation:
        translations.append(translation.read())

    hypotheses.append((model.transcribe(r"./Europarl-ST/es/en/test/%s/audio_clip_diarization.m4a" % audio, language="Spanish"))['text'])

  0%|          | 0/86 [00:00<?, ?it/s]

<p style="page-break-after:always;"></p>

Transcription hypotheses, references and translations are stored into a Pandas dataframe. Show the two first hypotheses.

In [4]:
data = pd.DataFrame(dict(hypothesis=hypotheses, reference=references, translation=translations))
pd.set_option('display.max_colwidth', None)
data.head(2)

,hypothesis,reference,translation
0,"El Atlético de Madrid, los aficionados e incluso la policía española, están siendo maltratados por la Federación Europea de Fútbol, pero el caso trasciéndelo particular. Pues esos órganos federativos agraban las anciones a quienes recurren a la justicia ordinaria. Esta concepción medieval, esta ley del embudo es incompatible con el derecho con las instituciones europeas, desde las que hemos de reaccionar y lo caberemos haciendo, pues esos señores medievales, arbitrarios de orca y cuchillo, andeponernos en línea con el respeto al derecho y las garantías procesales ordinarias de nuestra Europa. Nada más muchas veces.","El Atlético de Madrid , los aficionados e incluso la policía española están siendo maltratados por la Federación Europea de Fútbol . Pero el caso transciende lo particular , pues esos órganos federativos agravan las sanciones a quienes recurren a la justicia ordinaria .\nEsta concepción medieval , esta ley del embudo es incompatible con el Derecho y con las instituciones europeas , desde las que hemos de reaccionar . Lo acabaremos haciendo , pues esos señores medievales arbitrarios de horca y cuchillo han de ponerse en línea con el respeto al Derecho y las garantías procesales ordinarias de nuestra Europa .\n","Atlético Madrid, its fans and even the Spanish police are being mistreated by the Union of European Football Associations. However, the problem is wider than this as these federative bodies tend to increase sanctions when people resort to the ordinary courts.\nThis mediaeval concept of one law for me and another for you is contrary to our law and the European institutions. We must therefore react. In fact, we will end up having to react as these arbitrary mediaeval tyrants must abide by the law and the ordinary procedural guarantees of our Europe."
1,"Gracias, señor presidente, señor Camisario. El terrorismo subcenó en lo global y la actuación de la delincuencia grave realizada también, y por tanto los medios tienen que ser proporcionales y tienen que lucharse con eficacia. Yo he tomado buena nota de las respuestas quedado a las preguntas que eran oportunas. Es verdad que hay que seguir garantías, es verdad que es un tema delicado, pero no es bien oscierto que es absolutamente inexcusable montar, formar una respuesta globalizada y armonizada. Para algunos que el terrorismo les pillan un poco lejos, les preocupa más las garantías habituales, me voy a preocupar las habituales y las colectivas. Y es absolutamente necesario que empecemos por donde podamos. Si hemos empezado por el transporte aéreo, donde ya las compañías tienen esos datos, empecemos por ahí. Esijamos garantías, veamos cuál es el ámbito aplicación, empecemos por las relaciones de las transportas internacionales. Y ojo, tendremos que seguir por las grandes condariones porque los terroristas, muchas veces no vienen de fuera, si no quieren de adentro, que se lo preguntan a Estados Unidos y eso se preguntan a los demás que así es y así tendremos que plantearlo.","Señor Presidente , señor Comisario , el terrorismo es un fenómeno global y la actuación de la delincuencia grave organizada también , y por tanto los medios tienen que ser proporcionales y hay que luchar con eficacia .\nHe tomado buena nota de las respuestas que ha dado a las preguntas , y eran oportunas : es verdad que hay que exigir garantías , es verdad que es un tema delicado , pero no es menos cierto que es absolutamente inexcusable montar , formar una respuesta globalizada y armonizada .\nA algunos , a los que el terrorismo les queda un poco lejos , les preocupan más las garantías individuales . A mí me preocupan las individuales y las colectivas , y es absolutamente necesario que empecemos por donde podamos . Si hemos de empezar por el transporte aéreo , donde ya las compañías tienen esos datos , empecemos por ahí .\nExijamos garantías , veamos cuál es el ámbito de aplicación , empecemos por las relaciones de los transportes internacionales , y ¡ 

Hypotheses and references are normalized using the Whisper Basic text standardisation/normalization module

In [6]:
normalizer = BasicTextNormalizer()

data["hypothesis_clean"] = [normalizer(text) for text in data["hypothesis"]]
data["reference_clean"] = [normalizer(text) for text in data["reference"]]
data["translation_clean"] = [normalizer(text) for text in data["translation"]]
data.head(2)

,hypothesis,reference,translation,hypothesis_clean,reference_clean,translation_clean
0,"El Atlético de Madrid, los aficionados e incluso la policía española, están siendo maltratados por la Federación Europea de Fútbol, pero el caso trasciéndelo particular. Pues esos órganos federativos agraban las anciones a quienes recurren a la justicia ordinaria. Esta concepción medieval, esta ley del embudo es incompatible con el derecho con las instituciones europeas, desde las que hemos de reaccionar y lo caberemos haciendo, pues esos señores medievales, arbitrarios de orca y cuchillo, andeponernos en línea con el respeto al derecho y las garantías procesales ordinarias de nuestra Europa. Nada más muchas veces.","El Atlético de Madrid , los aficionados e incluso la policía española están siendo maltratados por la Federación Europea de Fútbol . Pero el caso transciende lo particular , pues esos órganos federativos agravan las sanciones a quienes recurren a la justicia ordinaria .\nEsta concepción medieval , esta ley del embudo es incompatible con el Derecho y con las instituciones europeas , desde las que hemos de reaccionar . Lo acabaremos haciendo , pues esos señores medievales arbitrarios de horca y cuchillo han de ponerse en línea con el respeto al Derecho y las garantías procesales ordinarias de nuestra Europa .\n","Atlético Madrid, its fans and even the Spanish police are being mistreated by the Union of European Football Associations. However, the problem is wider than this as these federative bodies tend to increase sanctions when people resort to the ordinary courts.\nThis mediaeval concept of one law for me and another for you is contrary to our law and the European institutions. We must therefore react. In fact, we will end up having to react as these arbitrary mediaeval tyrants must abide by the law and the ordinary procedural guarantees of our Europe.",el atlético de madrid los aficionados e incluso la policía española están siendo maltratados por la federación europea de fútbol pero el caso trasciéndelo particular pues esos órganos federativos agraban las anciones a quienes recurren a la justicia ordinaria esta concepción medieval esta ley del embudo es incompatible con el derecho con las instituciones europeas desde las que hemos de reaccionar y lo caberemos haciendo pues esos señores medievales arbitrarios de orca y cuchillo andeponernos en línea con el respeto al derecho y las garantías procesales ordinarias de nuestra europa nada más muchas veces,el atlético de madrid los aficionados e incluso la policía española están siendo maltratados por la federación europea de fútbol pero el caso transciende lo particular pues esos órganos federativos agravan las sanciones a quienes recurren a la justicia ordinaria esta concepción medieval esta ley del embudo es incompatible con el derecho y con las instituciones europeas desde las que hemos de reaccionar lo acabaremos haciendo pues esos señores medievales arbitrarios de horca y cuchillo han de ponerse en línea con el respeto al derecho y las garantías procesales ordinarias de nuestra europa,atlético madrid its fans and even the spanish police are being mistreated by the union of european football associations however the problem is wider than this as these federative bodies tend to increase sanctions when people resort to the ordinary courts this mediaeval concept of one law for me and another for you is contrary to our law and the european institutions we must therefore react in fact we will end up having to react as these arbitrary mediaeval tyrants must abide by the law and the ordinary procedural guarantees of our europe
1,"Gracias, señor presidente, señor Camisario. El terrorismo subcenó en lo global y la actuación de la delincuencia grave realizada también, y por tanto los medios tienen que ser proporcionales y tienen que lucharse con eficacia. Yo he tomado buena nota de las respuestas quedado a las preguntas que eran oportunas. Es verdad que hay que seguir garantías, es ve

Finally, we compute the transcription WER using [JIWER](https://openai.com/index/whisper/) which is a simple and fast python package to evaluate ASR performance.

In [7]:

wer = jiwer.wer(list(data["reference_clean"]), list(data["hypothesis_clean"]))

print(f"WER: {wer * 100:.2f} %")

WER: 14.83 %


All the data is stored into a file using 'csv' format

In [8]:
data.to_csv('L4.1_ASR_Whisper_Baseline_Europarl-ST.csv', encoding='utf-8')